# Image Captioning

This assignment is somewhat short.  We want you to spend your time on the project instead!

This assignment explores models connecting different modalities - exploring a connection between images and text.  By the time you're done this assignment, you'll:

* explored MS COCO captioning dataset
* investigated a few captioning techniques
* played with an image captioner


### Data

* Download the 2014 validation images, as well as their annotations from https://cocodataset.org/#download.  On a os x or linux, a command like ```curl http://images.cocodataset.org/zips/val2014.zip --output val2014.zip``` for each will do what you need.
* Unzip both files and name the corresponding directories `val2014` and `annotations` in this assignment directory.

### Explore the dataset
Look in the annotations directory.  Which file(s) contain the image captions?  Load those into memory here.  Your end goal is to generate a list of tuples for each of train2014 (you don't need the actual images to do this!) and val2014 i.e. ```[(391895, 'val2014/COCO_val2014_000000391895.jpg', 'A man with a red helmet on a small moped on a dirt road. '), ...]```

In [4]:
!pip3 install -U scikit-image
!pip3 install -U cython 
!pip3 install "git+https://github.com/philferriere/cocoapi.git#egg=pycocotools&subdirectory=PythonAPI"

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 12.4 MB 6.0 MB/s            
     |████████████████████████████████| 3.3 MB 31.0 MB/s            
     |████████████████████████████████| 4.4 MB 29.8 MB/s            
     |████████████████████████████████| 148 kB 55.3 MB/s            
     |████████████████████████████████| 1.6 MB 34.4 MB/s            
  Attempting uninstall: decorator
    Found existing installation: decorator 5.1.1
    Uninstalling decorator-5.1.1:
      Successfully uninstalled decorator-5.1.1
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 2.0 MB 9.5 MB/s            
Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/philferriere/cocoapi.git to /tmp/pip-install-cji6b680/pycocotools_2ed46ce729274317971b620aea4d7a9e
  Running command git clone --filter=blob:none -q https://git

In [9]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from pycocotools.coco import COCO
import pandas as pd

In [1]:
### YOUR CODE HERE

import json

!curl http://images.cocodataset.org/zips/val2014.zip --output val2014.zip
!curl http://images.cocodataset.org/annotations/annotations_trainval2014.zip --output annotations.zip

### END YOUR CODE

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 6337M  100 6337M    0     0  37.6M      0  0:02:48  0:02:48 --:--:-- 44.0M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  241M  100  241M    0     0  40.7M      0  0:00:05  0:00:05 --:--:-- 45.8M


In [5]:
import zipfile
#jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10

# specifying the zip file name
file_name = "val2014.zip"
  
# opening the zip file in READ mode
with zipfile.ZipFile(file_name, 'r') as zip:
    # printing all the contents of the zip file
    zip.printdir()
  
    # extracting all the files
    print('Extracting all the files now...')
    zip.extractall()
    print('Done!')

File Name                                             Modified             Size
val2014/                                       2014-08-16 01:15:20            0
val2014/COCO_val2014_000000324670.jpg          2014-08-16 00:41:54       134506
val2014/COCO_val2014_000000464263.jpg          2014-08-16 01:10:38       155585
val2014/COCO_val2014_000000526418.jpg          2014-08-16 01:13:08       102768
val2014/COCO_val2014_000000230593.jpg          2014-08-16 00:43:32       292537
val2014/COCO_val2014_000000186147.jpg          2014-08-16 00:58:00       154316
val2014/COCO_val2014_000000037149.jpg          2014-08-16 01:13:54       124209
val2014/COCO_val2014_000000284743.jpg          2014-08-16 01:09:08        68107
val2014/COCO_val2014_000000550691.jpg          2014-08-16 00:39:12       164686
val2014/COCO_val2014_000000515126.jpg          2014-08-16 00:55:48       220201
val2014/COCO_val2014_000000539419.jpg          2014-08-16 00:50:46       119589
val2014/COCO_val2014_000000401240.jpg   

val2014/COCO_val2014_000000452414.jpg          2014-08-16 00:56:14       150063
val2014/COCO_val2014_000000433192.jpg          2014-08-16 01:11:28        70662
val2014/COCO_val2014_000000467314.jpg          2014-08-16 00:30:24       101428
val2014/COCO_val2014_000000431431.jpg          2014-08-16 00:46:30       199425
val2014/COCO_val2014_000000568554.jpg          2014-08-16 00:40:16       175656
val2014/COCO_val2014_000000153656.jpg          2014-08-16 00:58:44       114257
val2014/COCO_val2014_000000527961.jpg          2014-08-16 00:41:08       187504
val2014/COCO_val2014_000000330720.jpg          2014-08-16 00:33:52       199989
val2014/COCO_val2014_000000447088.jpg          2014-08-16 01:09:30       148413
val2014/COCO_val2014_000000075886.jpg          2014-08-16 00:43:28       121537
val2014/COCO_val2014_000000209685.jpg          2014-08-16 00:42:32       176711
val2014/COCO_val2014_000000489091.jpg          2014-08-16 00:31:12        31783
val2014/COCO_val2014_000000170605.jpg   

val2014/COCO_val2014_000000499181.jpg          2014-08-16 00:42:02       175596
val2014/COCO_val2014_000000259253.jpg          2014-08-16 00:41:26       282426
val2014/COCO_val2014_000000164924.jpg          2014-08-16 01:03:06       205120
val2014/COCO_val2014_000000195648.jpg          2014-08-16 00:39:30       390349
val2014/COCO_val2014_000000342515.jpg          2014-08-16 00:35:30       265391
val2014/COCO_val2014_000000337160.jpg          2014-08-16 00:56:46       190679
val2014/COCO_val2014_000000013882.jpg          2014-08-16 00:33:56       188128
val2014/COCO_val2014_000000066711.jpg          2014-08-16 00:44:04       116764
val2014/COCO_val2014_000000528720.jpg          2014-08-16 01:00:06        82904
val2014/COCO_val2014_000000100811.jpg          2014-08-16 00:40:30       103540
val2014/COCO_val2014_000000120285.jpg          2014-08-16 01:08:02       127720
val2014/COCO_val2014_000000005879.jpg          2014-08-16 00:32:20       136783
val2014/COCO_val2014_000000350488.jpg   

val2014/COCO_val2014_000000096241.jpg          2014-08-16 00:47:58       240682
val2014/COCO_val2014_000000404071.jpg          2014-08-16 00:44:58       298001
val2014/COCO_val2014_000000578553.jpg          2014-08-16 01:05:08        50019
val2014/COCO_val2014_000000229858.jpg          2014-08-16 00:46:08       320240
val2014/COCO_val2014_000000091670.jpg          2014-08-16 00:42:22       232505
val2014/COCO_val2014_000000234251.jpg          2014-08-16 01:07:14        95572
val2014/COCO_val2014_000000109231.jpg          2014-08-16 00:35:52       240058
val2014/COCO_val2014_000000232865.jpg          2014-08-16 00:55:24        92683
val2014/COCO_val2014_000000009813.jpg          2014-08-16 00:54:32       229245
val2014/COCO_val2014_000000112798.jpg          2014-08-16 00:46:48       130819
val2014/COCO_val2014_000000520289.jpg          2014-08-16 00:47:46       156209
val2014/COCO_val2014_000000500543.jpg          2014-08-16 00:50:12        92873
val2014/COCO_val2014_000000064196.jpg   

val2014/COCO_val2014_000000281101.jpg          2014-08-16 01:05:14       135113
val2014/COCO_val2014_000000161557.jpg          2014-08-16 00:52:48       160388
val2014/COCO_val2014_000000258290.jpg          2014-08-16 01:14:46       132117
val2014/COCO_val2014_000000075612.jpg          2014-08-16 01:06:40       104123
val2014/COCO_val2014_000000456127.jpg          2014-08-16 00:39:22       221746
val2014/COCO_val2014_000000296014.jpg          2014-08-16 00:49:14       267734
val2014/COCO_val2014_000000026907.jpg          2014-08-16 00:54:54        94507
val2014/COCO_val2014_000000556518.jpg          2014-08-16 01:09:46       151375
val2014/COCO_val2014_000000356742.jpg          2014-08-16 01:03:02        34522
val2014/COCO_val2014_000000147027.jpg          2014-08-16 00:36:18       134678
val2014/COCO_val2014_000000214241.jpg          2014-08-16 00:54:10       228427
val2014/COCO_val2014_000000437664.jpg          2014-08-16 01:03:32       232693
val2014/COCO_val2014_000000376365.jpg   

val2014/COCO_val2014_000000300034.jpg          2014-08-16 00:32:28       129326
val2014/COCO_val2014_000000117432.jpg          2014-08-16 00:54:00       251645
val2014/COCO_val2014_000000032391.jpg          2014-08-16 00:34:32       148840
val2014/COCO_val2014_000000550400.jpg          2014-08-16 00:59:08       266024
val2014/COCO_val2014_000000277961.jpg          2014-08-16 00:53:08       330341
val2014/COCO_val2014_000000263223.jpg          2014-08-16 00:56:34       162634
val2014/COCO_val2014_000000488697.jpg          2014-08-16 00:52:32       169818
val2014/COCO_val2014_000000512911.jpg          2014-08-16 00:50:58       125180
val2014/COCO_val2014_000000221708.jpg          2014-08-16 00:30:48       209595
val2014/COCO_val2014_000000283977.jpg          2014-08-16 00:49:26       254641
val2014/COCO_val2014_000000226588.jpg          2014-08-16 00:57:50       164478
val2014/COCO_val2014_000000483328.jpg          2014-08-16 00:35:34        98628
val2014/COCO_val2014_000000200775.jpg   

val2014/COCO_val2014_000000527102.jpg          2014-08-16 00:29:32       310144
val2014/COCO_val2014_000000335041.jpg          2014-08-16 00:57:30       243665
val2014/COCO_val2014_000000245701.jpg          2014-08-16 01:10:04       123889
val2014/COCO_val2014_000000328238.jpg          2014-08-16 00:51:56       175705
val2014/COCO_val2014_000000027932.jpg          2014-08-16 00:43:18        75978
val2014/COCO_val2014_000000333303.jpg          2014-08-16 01:11:56        86133
val2014/COCO_val2014_000000005038.jpg          2014-08-16 01:04:16       190622
val2014/COCO_val2014_000000053410.jpg          2014-08-16 01:07:32       116817
val2014/COCO_val2014_000000329957.jpg          2014-08-16 00:30:34       172778
val2014/COCO_val2014_000000363048.jpg          2014-08-16 00:34:42       206292
val2014/COCO_val2014_000000177173.jpg          2014-08-16 00:38:58       170906
val2014/COCO_val2014_000000381109.jpg          2014-08-16 00:57:32       171604
val2014/COCO_val2014_000000208174.jpg   

val2014/COCO_val2014_000000482362.jpg          2014-08-16 01:01:46        88850
val2014/COCO_val2014_000000503926.jpg          2014-08-16 01:04:46       118677
val2014/COCO_val2014_000000397041.jpg          2014-08-16 01:00:46       197252
val2014/COCO_val2014_000000222313.jpg          2014-08-16 00:29:26       145178
val2014/COCO_val2014_000000255592.jpg          2014-08-16 00:48:24       141208
val2014/COCO_val2014_000000125322.jpg          2014-08-16 01:12:14        92030
val2014/COCO_val2014_000000472424.jpg          2014-08-16 00:56:50       221431
val2014/COCO_val2014_000000051272.jpg          2014-08-16 00:46:22       130117
val2014/COCO_val2014_000000130076.jpg          2014-08-16 01:07:22        86932
val2014/COCO_val2014_000000495612.jpg          2014-08-16 00:41:24       118386
val2014/COCO_val2014_000000534373.jpg          2014-08-16 01:01:32       223271
val2014/COCO_val2014_000000022257.jpg          2014-08-16 00:38:24       284126
val2014/COCO_val2014_000000227134.jpg   

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Done!


In [6]:
with zipfile.ZipFile("annotations.zip","r") as annot_zip:
    annot_zip.extractall()

In [23]:
### YOUR CODE HERE
import json

#extract file name for relevant image_id
def search(id, data):
    return [element['file_name'] for element in data if element['id'] == id][0]

train2014 = []
val2014 = []

train_captions_filepath = 'annotations/captions_train2014.json'
val_captions_filepath = 'annotations/captions_val2014.json'

train_data_file = open(train_captions_filepath)
train_data = json.load(train_data_file)

for i in train_data['annotations']:
    train2014.append((i['image_id'], 'train2014/'+ str(search(i['image_id'], train_data['images'])), i['caption']))
    
train_data_file.close()
    
val_data_file = open(val_captions_filepath)
val_data = json.load(val_data_file)

for i in val_data['annotations']:
    val2014.append((i['image_id'], 'val2014/'+ str(search(i['image_id'], val_data['images'])), i['caption']))
    
val_data_file.close()


### END YOUR CODE

In [25]:
train2014_temp = [tuple(x) for x in train2014]
val2014_temp = [tuple(x) for x in val2014]

In [28]:
assert (479495,
  'train2014/COCO_train2014_000000479495.jpg',
  'A bicycle is parked by a bench at night.') in train2014_temp
assert (203564,
  'val2014/COCO_val2014_000000203564.jpg',
  'A black metal bicycle with a clock inside the front wheel.') in val2014_temp

In [27]:
assert (479495,
  'train2014/COCO_train2014_000000479495.jpg',
  'A bicycle is parked by a bench at night.') in train2014
assert (203564,
  'val2014/COCO_val2014_000000203564.jpg',
  'A black metal bicycle with a clock inside the front wheel.') in val2014

In [29]:
train2014_df = pd.DataFrame(train2014, columns=['id', 'file', 'caption'])

In [34]:
image_captions = len(train2014_df.id.unique())
captions_per_image = train2014_df.groupby('id')['caption'].count().mean()
split_words_vocab = len(np.unique(' '.join([i for i in train2014_df['caption']]).split()))

print('Images that we have captions for in training: ', image_captions)
print('Captions per image: ', captions_per_image)
print('Words in vocab (split on whitespace) ', split_words_vocab)

Images that we have captions for in training:  82783
Captions per image:  5.0023917954169335
Words in vocab (split on whitespace)  44535


### Questions (Part A)

1. How many images do we have captions for in the training set?
2. How many captions do we have per image (rounded to nearest integer)?
3. If you just split on whitespace and do nothing else, how many words are in the vocabulary?

Given your answer to 3, think about what you might need to do with that few examples and that large of a vocabulary?  (Use pretrained embeddings from a massive semi-supervised dataset, keeping only the top-k tokens, smarter tokenization, ...).  You don't need to write your answer anywhere, but given how often these problems arise in NLP, you should be feeling more confident at this point in the course how to handle these situations!

## Let's look at some of the examples!

In [33]:
import matplotlib.pyplot as plt
from keras.preprocessing.image import load_img, img_to_array
import random

samples = 5
n = 1
fig = plt.figure(figsize=(10,20))
for i in range(samples):
    image_caption = val2014[random.randint(0, len(val2014))]
    image_load = load_img(image_caption[1])
    
    ax = fig.add_subplot(samples,2,n,xticks=[],yticks=[])
    ax.imshow(image_load)
    n += 1
    
    ax = fig.add_subplot(samples,2,n)
    plt.axis('off')
    ax.plot()
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.text(0, 0, image_caption[2], fontsize=20)
    n += 1
plt.show()

# Foundational image captioning papers

## Show & Tell

[Show and Tell: A Neural Image Caption Generator](https://arxiv.org/pdf/1411.4555.pdf) was the first step towards neural image captioning.  Fundamentally it is a encoder-decoder scheme similar to what we've seen in class.  Concretely, it uses the CNN structure of a (at the time) state of the art image classification CNN as the encoder and it uses a LSTM as a decoder.  As in the generation models in class, it continues to generate text until a special "stop" token is emitted.  After **reading** the paper, answer the following questions:

### Questions (Part B)

1.  What parts of the CNN were trained?
2.  What was the biggest conern when deciding?
3.  How was the encoded image representation input into the decoder?
4.  Given we are "translating" from an image to a caption (without a length constraint), which evaluation metric did the authors determine was reasonable for a top line metric?
5.  What beam width is equivalent to one where you select the highest probability word in each decoding step?
6.  How many points of quality did using a beam search of 20 provide versus a greedy search?
7.  Where did the authors get their word embeddings from?
8.  Approximately how often were there novel sentences in the top-15 generated text candidates, despite overfitting issues?

## Deep visual alignment

[Deep Visual-Semantic Alignments for Generating Image Descriptions](https://cs.stanford.edu/people/karpathy/cvpr2015.pdf) is a fun read for which we will ask no questions.  Its critical insights are around understanding an image as a composition of regions, and building upon that understanding to construct both a caption for the whole image, but labels for its consistuent parts.

## Show, Attend & Tell

[Show, Attend & Tell](https://arxiv.org/pdf/1502.03044.pdf) applies the same "provide the decoder more context, as directly as possible" trick we've seen over the course: adding attention.  After **skimming** the paper, answer the following questions:

### Questions (Part C)

1. What is the attention over?
2. What do the figures with highlight shading represent in Figures 2, 3 and 5?

# Exploring a MS COCO captioner

There are many examples of image captioners ML engineers have built on the MS COCO dataset you explored. [This one](https://replicate.com/rmokady/clip_prefix_caption) uses a (more) modern large language model as its decoder, GPT-2.  

* **Explore** the samples and play with using beam search and not.  What do you notice?

This is an example from the Show & Tell paper of a low-quality caption (see figure 5).  The GPT-2 model proposes "the car that person drove to the hospital." vs. "A yellow school bus parked in a parking lot" from the original paper. ![Misclassified](littlecar.png) 